Početak 2020-02-18

In [8]:
import pandas as pd
from pandas import DataFrame as df
import numpy as np
from google.colab import files
import datetime
import copy
import csv

In [2]:
from pandas.core.frame import DataFrame
start_date="2021-10-05"
end_date="2021-10-10"
wunderground_station_id="IPTUJ11"
station_id="DUDU1234"

range_of_dates=pd.date_range(start=start_date, end=end_date)
range_of_dates=list(map(lambda x: str(x)[:10],range_of_dates))

In [3]:
def colecting_data(range_of_dates,wunderground_station_id):
  final_file=pd.DataFrame(columns=['DateTime', 'Date','Time', 'Temperature', 'Dew Point', 'Humidity', 'Wind', 'Speed', 'Gust', 'Pressure', 'Precip. Rate.', 'Precip. Accum.', 'UV', 'Solar'])
  for measurement_date in range_of_dates:
    url=f"https://www.wunderground.com/dashboard/pws/{wunderground_station_id}/table/{measurement_date}/{measurement_date}/daily"
    dfs = pd.read_html(url,match="Precip. Rate.")
    data = pd.concat(dfs)
    data.dropna(inplace = True)
    data.insert(0, 'Date', measurement_date)
    final_file=pd.concat([final_file, data], ignore_index=True)

  return final_file


In [4]:
raw_data=colecting_data(range_of_dates, wunderground_station_id)
raw_data.to_csv(f"{wunderground_station_id}_{start_date}_{end_date}")

In [5]:
data=copy.deepcopy(raw_data)

In [6]:
data=data.replace(to_replace=r'°F', value='', regex=True)
data=data.replace(to_replace=r'°%', value='', regex=True)
data=data.replace(to_replace=r'°in', value='', regex=True)
data=data[['Date', 'Time', 'Temperature', 'Dew Point', 'Humidity', 'Pressure', 'Precip. Accum.']]
data['Temperature']=data['Temperature'].map(lambda x: round((((float(x)-32) * 5)/9),1))
data['Dew Point']=data['Dew Point'].map(lambda x: round((((float(x)-32) * 5)/9),1))
data['Pressure']=data['Pressure'].map(lambda x: round((float(x)*33.8639),1))
data['Precip. Accum.']=data['Precip. Accum.'].map(lambda x: round((float(x)*25.4),1))
data.insert(0, 'UTC DateTime', '')
data['UTC DateTime'] = (pd.to_datetime(data['Date'] + " " + data['Time'], format='%Y-%m-%d %I:%M %p')-datetime.timedelta(hours=2)).dt.strftime('%Y%m%d%H%M')
data['Date']=data['UTC DateTime'].map(lambda x: x[:-4])
data['Time']=data['UTC DateTime'].map(lambda x: x[8:])
data



,UTC DateTime,Date,Time,Temperature,Dew Point,Humidity,Pressure,Precip. Accum.
0,202110042219,20211004,2219,14.0,12.6,91,1011.2,0.0
1,202110042220,20211004,2220,13.6,11.9,90,1011.2,0.0
2,202110042229,20211004,2229,13.5,12.3,92,1011.2,0.0
3,202110042232,20211004,2232,13.4,12.2,92,1011.2,0.0
4,202110042244,20211004,2244,13.2,11.9,92,1011.2,0.0
...,...,...,...,...,...,...,...,...
1308,202110102131,20211010,2131,6.6,5.6,94,1017.3,15.0
1309,202110102139,20211010,2139,6.7,5.6,93,1017.3,15.0
1310,202110102143,20211010,2143,6.7,5.6,93,1017.3,15.2
1311,202110102154,20211010,2154,6.7,5.6,93,1017.3,15.2


#Precipitation

In [ ]:
daily_precipitation=data['Precip. Accum.'].sum()
if daily_precipitation < 0.5:
  daily_precipitation= 'T'



```
# This is formatted as code
```



In [29]:
precipitation_data=data.pivot_table(index='Date', values='Precip. Accum.', aggfunc=np.max)
precipitation_data['Precip. Accum.']



,Precip. Accum.
Date,
20211004,0.0
20211005,0.0
20211006,58.9
20211007,44.7
20211008,1.3
20211009,0.0
20211010,15.2


In [ ]:
for measurement_time, temperature in zip(data['Time'], data['Temperature']):
    print(f"PR {station_id} 1 {measurement_time[:-4]}2155 {daily_precipitation} 1 X X")


#Temperature

In [26]:
def export_temperature(data):
  with open("temperature.txt", "w") as f1:       
    for measurement_time, temperature in zip(data['Time'], data['Temperature']):
      f1.write(f"ATMNN {station_id} 01 {measurement_time} {temperature}\n")

#Pressure

In [27]:
def export_pressure(data):
  with open("pressure.txt", "w") as f1:       
    for measurement_time, pressure in zip(data['Time'], data['Pressure']):
      f1.write(f"BP {station_id} 01 {measurement_time} 1 {pressure}\n")

#Humidity

In [28]:
def export_humidity(data):
  with open("humidity.txt", "w") as f1:       
    for measurement_time, temperature, humidity in zip(data['Time'], data['Temperature'], data['Humidity']):
      f1.write(f"RH {station_id} 01 {measurement_time} D {temperature} X {humidity}\n")

#Other

In [ ]:
files.download()